In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras

assert tf.__version__ == '1.12.0'
assert keras.__version__ == '2.1.6-tf'

if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Used GPU at: {}'.format(device_name))

In [2]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.models as models

import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

## Evaluate Functions

In [3]:
def decode_prediction(Pred, Top=1):
    res = []
    for pred in Pred:
        scores = [(pred[i], i) for i in range(120)]
        scores.sort(reverse=1)
        res.append(scores[:Top])
    return res

def calculate_accuracy(Pred, Lbl, Top):
    idx, cnt = 0, 0
    for res in decode_prediction(Pred, Top):
        lbls = [res[i][1] for i in range(Top)]
        if Lbl[idx] in lbls:
            cnt += 1
        idx += 1
    return cnt / len(Pred)

## Prepare the testing data

In [4]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Test'

In [20]:
ImgGen_Inception = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input)
ImgGen_DenseNet  = image.ImageDataGenerator(preprocessing_function=densenet.preprocess_input)

## InceptionV3 Evaluation

In [6]:
model_inceptionV3 = models.load_model('./models/inceptionv3_catdfc120_258_8064')

In [19]:
# Generate Random Subsets 
Test_Inception  = ImgGen_Inception.flow_from_directory(
                        join(prefix, imgfix), 
                        target_size=(224, 224), 
                        class_mode='sparse', 
                        batch_size=32, 
                        shuffle=True)

Found 8580 images belonging to 120 classes.


In [8]:
# pick up the first batch from the generator
pred = model_inceptionV3.predict_on_batch(Test_Inception[0][0])
lbls = Test_Inception[0][1]

print(calculate_accuracy(pred, lbls, Top=1))

0.78125


In [9]:
## Display the Misclassified Samples
Y = decode_prediction(pred)
for k in [Y[i][0][1]==lbls[i] for i in range(32)]:
    if k: continue
    # display the image here

## DenseNet121 Evaluation

In [16]:
model_densenet121 = models.load_model('./models/densenet121_dfc120_finetune_9_8141')

In [21]:
# Generate Random Subsets 
Test_DenseNet = ImgGen_DenseNet.flow_from_directory(
                        join(prefix, imgfix), 
                        target_size=(224, 224), 
                        class_mode='sparse', 
                        batch_size=32, 
                        shuffle=True)

Found 8580 images belonging to 120 classes.


In [22]:
# pick up the first batch from the generator
pred = model_densenet121.predict_on_batch(Test_DenseNet[0][0])
lbls = Test_DenseNet[0][1]

print(calculate_accuracy(pred, lbls, Top=1))

0.875


In [23]:
## Display the Misclassified Samples
Y = decode_prediction(pred)
for k in [Y[i][0][1]==lbls[i] for i in range(32)]:
    if k: continue
    # display the image here

## VGG Evaluation

Since the model of VGG19 is too large, approx. 1GB, which exceed the limits of github. 

Therefore we will skip evaluate the VGG model.

In [ ]:
# model_vgg = models.load_model('vgg19_dfc120_40_6751')
# res_vgg = model_vgg.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)

# print('Losses of the fine tuned VGG model',res_vgg[0])
# print('Accuracy of the fine tuned VGG model',res_vgg[1])
